In [38]:
import os
import re
import sys
sys.path.append('./')
from collections import defaultdict
import xml.etree.ElementTree as ET
from xbrl import XBRLParser
from xbrl_parser import XbrlParser
import pandas as pd
import requests

namespaces = {
    'link': 'http://www.xbrl.org/2003/linkbase',
    'xml':'http://www.w3.org/XML/1998/namespace',
    'xlink':'http://www.w3.org/1999/xlink',
    'xsi':'http://www.w3.org/2001/XMLSchema-instance'
}


base_path = os.getcwd()+'/xbrl_files/'
_dir = 'S100APUV/XBRL/PublicDoc/'
xbrl_filename = base_path+_dir+'jpcrp030000-asr-001_E02066-000_2017-03-31_01_2017-06-29.xbrl'

# get data
xp = XbrlParser(xbrl_filename)

print('getting data...')
xbrl_persed = xp.parse_xbrl(namespaces)
print('done')

df_xbrl_facts = xbrl_persed['facts'] # 金額の定義及び文書情報の定義情報
df_xbrl_labels = xbrl_persed['labels'] # 名称リンク情報
df_xbrl_presentation = xbrl_persed['presentation'] # 表示リンク情報

dat_fi = pd.merge(df_xbrl_labels, df_xbrl_facts, on='element_id', how='inner')
# specify duration
dat_fi_cyi = dat_fi.loc[dat_fi.context_ref == 'CurrentYearInstant'] # 当期 時点


# rol_ConsolidatedBalanceSheet
df_xbrl_ps_cbs = df_xbrl_presentation.loc[df_xbrl_presentation.role_id.str
                                             .contains('rol_ConsolidatedBalanceSheet'), :]

from decimal import Decimal
import json

series = []

currentassets = []
xp.recursive(currentassets, df_xbrl_ps_cbs, 'jppfs_cor_currentassetsabstract')
currentassets_coa = xp.get_specific_account_name(dat_fi_cyi, currentassets).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in currentassets_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [int(value), None],
        'stack': 'Credit'
    })

noncurrentassets = []
xp.recursive(noncurrentassets, df_xbrl_ps_cbs, 'jppfs_cor_noncurrentassetsabstract')
noncurrentassets_coa = xp.get_specific_account_name(dat_fi_cyi, noncurrentassets).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in noncurrentassets_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [int(value), None],
        'stack': 'Credit'
    })


liabilities = []
xp.recursive(liabilities, df_xbrl_ps_cbs, 'jppfs_cor_liabilitiesabstract')
liabilities_coa = xp.get_specific_account_name(dat_fi_cyi, liabilities).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in liabilities_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [int(value), None],
        'stack': 'Debit'
    })

netassets = []
xp.recursive(netassets, df_xbrl_ps_cbs, 'jppfs_cor_netassetsabstract')
netassets_coa = xp.get_specific_account_name(dat_fi_cyi, netassets).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in netassets_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [int(value), None],
        'stack': 'Debit'
    })
print(json.dumps({"series": series}, ensure_ascii=False))

getting data...
done
{"series": [{"name": "現金及び預金", "data": [21819012000, null], "stack": "Credit"}, {"name": "受取手形及び売掛金", "data": [16508363000, null], "stack": "Credit"}, {"name": "有価証券", "data": [3182261000, null], "stack": "Credit"}, {"name": "商品及び製品", "data": [8723172000, null], "stack": "Credit"}, {"name": "仕掛品", "data": [193869000, null], "stack": "Credit"}, {"name": "原材料及び貯蔵品", "data": [1124072000, null], "stack": "Credit"}, {"name": "繰延税金資産", "data": [1885654000, null], "stack": "Credit"}, {"name": "その他", "data": [782137000, null], "stack": "Credit"}, {"name": "貸倒引当金", "data": [-10572000, null], "stack": "Credit"}, {"name": "流動資産", "data": [54207971000, null], "stack": "Credit"}, {"name": "建物及び構築物（純額）", "data": [2083660000, null], "stack": "Credit"}, {"name": "機械装置及び運搬具（純額）", "data": [1174435000, null], "stack": "Credit"}, {"name": "土地", "data": [1130232000, null], "stack": "Credit"}, {"name": "建設仮勘定", "data": [15105000, null], "stack": "Credit"}, {"name": "その他（純額）", "data": [7

In [40]:
%%HTML
    <script src="https://code.highcharts.com/highcharts.js"></script>
    <div id="container" style="width:100%; height:400px;"></div>
    <script>
        new Highcharts.chart('container', {
            chart: {
                type: 'column'

            },

            title: {
                text: '会社名'
            },

            xAxis: {
                categories: ['Balance Sheets', 'Income Statement']
            },

            yAxis: {
                allowDecimals: false,
                min: 0,
                title: {
                    text: 'Yen'
                }
            },

            tooltip: {
                formatter: function () {
                    return '<b>' + this.x + '</b><br/>' +
                        this.series.name + ': ' + this.y;
                }
            },

            plotOptions: {
                column: {
                    stacking: 'normal',
                },
                series: {
                    pointPadding: 0,
                    borderWidth: 0,
                    shadow: false,
                }
            },

            series: [{
                name: '現金預金等',
                data: [1169409, null],
                stack: 'Credit'
            }, {
                name: '売掛金',
                data: [46840, null],
                stack: 'Credit'
            }, {
                name: '棚卸資産',
                data: [2627138, null],
                stack: 'Credit'
            }, {
                name: 'その他流動資産',
                data: [405844, null],
                stack: 'Credit'
            }, {
                name: '有形固定資産',
                data: [17990, null],
                stack: 'Credit'
            }, {
                name: '無形固定資産',
                data: [49806, null],
                stack: 'Credit'
            }, {
                name: 'その他固定資産',
                data: [19854, null],
                stack: 'Credit'
            }, {
                name: '買掛金',
                data: [4372, null],
                stack: 'Dedit'
            }, {
                name: '有利子負債',
                data: [521575, null],
                stack: 'Dedit'
            }, {
                name: 'その他流動負債',
                data: [864756, null],
                stack: 'Dedit'
            }, {
                name: 'その他固定負債',
                data: [0, null],
                stack: 'Dedit'
            }, {
                name: '純資産',
                data: [2950550, null],
                stack: 'Dedit'
            }, {
                name: '営業原価',
                data: [null, 4596029],
                stack: 'Credit'
            }, {
                name: '販管費',
                data: [null, 710028],
                stack: 'Credit'
            }, {
                name: '営業利益',
                data: [null, 1027949],
                stack: 'Credit'
            }, {
                name: '売上',
                data: [null, 6334008],
                stack: 'Dedit'
            }],
      });
    </script>

In [61]:
template="""
    <script src="https://code.highcharts.com/highcharts.js"></script>
    <div id="container" style="width:100%; height:400px;"></div>
    <script>
        new Highcharts.chart('container', {
            chart: {
                type: 'column'

            },

            title: {
                text: '会社名'
            },

            xAxis: {
                categories: ['Balance Sheets', 'Income Statement']
            },

            yAxis: {
                allowDecimals: false,
                min: 0,
                title: {
                    text: 'Yen'
                }
            },

            tooltip: {
                formatter: function () {
                    return '<b>' + this.x + '</b><br/>' +
                        this.series.name + ': ' + this.y;
                }
            },

            plotOptions: {
                column: {
                    stacking: 'normal',
                },
                series: {
                    pointPadding: 0,
                    borderWidth: 0,
                    shadow: false,
                }
            },

            series: [{
                name: '現金預金等',
                data: [1169409, null],
                stack: 'Credit'
            }, {
                name: '売掛金',
                data: [46840, null],
                stack: 'Credit'
            }, {
                name: '棚卸資産',
                data: [2627138, null],
                stack: 'Credit'
            }, {
                name: 'その他流動資産',
                data: [405844, null],
                stack: 'Credit'
            }, {
                name: '有形固定資産',
                data: [17990, null],
                stack: 'Credit'
            }, {
                name: '無形固定資産',
                data: [49806, null],
                stack: 'Credit'
            }, {
                name: 'その他固定資産',
                data: [19854, null],
                stack: 'Credit'
            }, {
                name: '買掛金',
                data: [4372, null],
                stack: 'Dedit'
            }, {
                name: '有利子負債',
                data: [521575, null],
                stack: 'Dedit'
            }, {
                name: 'その他流動負債',
                data: [864756, null],
                stack: 'Dedit'
            }, {
                name: 'その他固定負債',
                data: [0, null],
                stack: 'Dedit'
            }, {
                name: '純資産',
                data: [2950550, null],
                stack: 'Dedit'
            }, {
                name: '営業原価',
                data: [null, 4596029],
                stack: 'Credit'
            }, {
                name: '販管費',
                data: [null, 710028],
                stack: 'Credit'
            }, {
                name: '営業利益',
                data: [null, 1027949],
                stack: 'Credit'
            }, {
                name: '売上',
                data: [null, 6334008],
                stack: 'Dedit'
            }],
      });
      </script>
"""

HTML(template)
